In [1]:
!pip install transformers
!pip install datasets
!pip install SentencePiece
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.2 MB/s eta 0:00:00


In [1]:
from datasets import load_dataset
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
# Load pre-trained Pegasus model and tokenizer
model_name = 'google/pegasus-cnn_dailymail' #요약 데이터
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Load and preprocess the dataset
train_dataset = load_dataset('cnn_dailymail', '3.0.0', split='train[:50]')  # Load a subset of the "cnn_dailymail" dataset

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


In [12]:
train_dataset

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 50
})

## 예시 기사와 article 본문과 highlights 요약 기사 전달
## truncation : 자르기 ok
## padding : longest ->
## labels? -> tokenizer된후 target의 id에서 가져온 것
### tokenizer로 변환후 input_ids 를 살펴보면 tokenizer된 vector 존재

In [4]:
def preprocess_function(examples):
    inputs = tokenizer(examples['article'], truncation=True, padding='longest')
    targets = tokenizer(examples['highlights'], truncation=True, padding='longest')
    inputs['labels'] = targets['input_ids']
    return inputs

## Preprocessing Function
### Map 사용하여 적용 , batched True


In [5]:
train_dataset = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## 훈련인자 설정
### {}TrainingArgument
### {}Trainer


In [6]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=1,
    save_steps=500,
    save_total_limit=3,
    eval_steps=100,
    logging_steps=100,
    logging_dir='./logs',
    learning_rate=1e-5,
    warmup_steps=500,
    weight_decay=0.01,
)

In [7]:
# Create the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
)

In [8]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=50, training_loss=4.3465509033203125, metrics={'train_runtime': 46.4639, 'train_samples_per_second': 1.076, 'train_steps_per_second': 1.076, 'total_flos': 144473220710400.0, 'train_loss': 4.3465509033203125, 'epoch': 1.0})

In [9]:
model.save_pretrained("path/to/save/directory")

## Memory issue의 경우 Runtime 재실행후 아래 코드만 실행

In [10]:
from transformers import PegasusForConditionalGeneration

model = PegasusForConditionalGeneration.from_pretrained("path/to/save/directory")

In [20]:
input_text = ["i hate you"]
input_encoding = tokenizer(input_text, truncation=True, padding="longest", max_length=512, return_tensors="pt")

# Generate predictions
output = model.generate(input_encoding["input_ids"])
predicted_text = tokenizer.batch_decode(output, skip_special_tokens=True)

In [21]:
predicted_text

['i hate you.<n>i hate you.<n>i hate you.<n>i hate you.<n>i hate you.<n>i hate you.<n>i hate you.']